In [1]:
import os

os.chdir('../../Datasets')
os.getcwd()

'/Users/brunogutierrez/workspace/tese/Datasets'

In [2]:
# Standard libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import lines

from nltk.tokenize import sent_tokenize
# import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
raw_path = os.getcwd() + '/Olist/'
order_reviews = pd.read_csv(raw_path + 'olist_order_reviews_dataset.csv')
products = pd.read_csv(raw_path + 'olist_products_dataset.csv')
orders = pd.read_csv(raw_path + 'olist_orders_dataset.csv')
order_items = pd.read_csv(raw_path + 'olist_order_items_dataset.csv')

In [4]:
# Reading all the files
# raw_path = os.getcwd() + '/Olist/'
# customer = pd.read_csv(raw_path + 'olist_customers_dataset.csv')
# geolocation = pd.read_csv(raw_path + 'olist_geolocation_dataset.csv')
# order_items = pd.read_csv(raw_path + 'olist_order_items_dataset.csv')
# order_payments = pd.read_csv(raw_path + 'olist_order_payments_dataset.csv')
# order_reviews = pd.read_csv(raw_path + 'olist_order_reviews_dataset.csv')
# products = pd.read_csv(raw_path + 'olist_products_dataset.csv')
# sellers = pd.read_csv(raw_path + 'olist_sellers_dataset.csv')

### Quantas reviews possuímos?

In [5]:
order_reviews.sample()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
52501,02018114775e6e62168d362ef7650304,8689805b7502503017ca112059637a7f,5,NaN,NaN,2018-01-20 00:00:00,2018-01-23 21:55:39


In [6]:
order_reviews[['review_score']].describe()
# A maioria possui valor NA
order_reviews[['review_comment_title', 'review_comment_message']].isna().sum()

,review_score
count,100000.000000
mean,4.070890
std,1.359663
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


review_comment_title      88285
review_comment_message    58247
dtype: int64

In [7]:
order_reviews[['review_comment_title', 'review_comment_message']].sample()

,review_comment_title,review_comment_message
5174,PESSIMO,Paguei por um frete onde os prazos não foram s...


In [8]:
# Contar letras

In [9]:
def count_char(text):
    return len(text) if isinstance(text, str) else 0

In [10]:
order_reviews['n_char_title'] = order_reviews['review_comment_title'].apply(count_char)
order_reviews['n_char_text'] = order_reviews['review_comment_message'].apply(count_char)


order_reviews['n_char_title'].describe()
order_reviews['n_char_text'].describe()

count    100000.000000
mean          1.405100
std           4.426203
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          26.000000
Name: n_char_title, dtype: float64

count    100000.000000
mean         29.243230
std          49.455495
min           0.000000
25%           0.000000
50%           0.000000
75%          43.000000
max         208.000000
Name: n_char_text, dtype: float64

In [11]:
# contar frases e palavras

In [27]:
def count_phrases(text):
    if type(text) == str:
        return len(sent_tokenize(text, language='portuguese'))
    return 0

def words_count(x):
    if type(x)==str:
        return len(x.split())
    else:
        return np.nan

In [13]:
order_reviews['n_sentences'] = order_reviews['review_comment_message'].apply(count_phrases)
order_reviews[order_reviews['n_sentences']>0]['n_sentences'].describe()

count    41726.000000
mean         1.427959
std          0.791097
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         32.000000
Name: n_sentences, dtype: float64

### Dropar linhas sem reviews

In [14]:
# número de reviews
# order_reviews = order_reviews.dropna(subset=['review_comment_title', 'review_comment_message'], how='all')
order_reviews = order_reviews.dropna(subset=['review_comment_message'])
order_reviews.shape
# order_reviews['full_review'] = order_reviews['review_comment_title'] + '.' + order_reviews['review_comment_message']

(41753, 10)

### Número de frases por Review

In [15]:
order_reviews['n_sentences'] = order_reviews['review_comment_message'].apply(count_phrases)
order_reviews['n_sentences'].describe()

count    41753.000000
mean         1.427035
std          0.791674
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         32.000000
Name: n_sentences, dtype: float64

### Palavras por Review

In [28]:
order_reviews['n_words'] = order_reviews['review_comment_message'].apply(words_count)
order_reviews['n_words'].describe()

count    41753.000000
mean        11.913419
std          9.704207
min          0.000000
25%          5.000000
50%          9.000000
75%         17.000000
max         45.000000
Name: n_words, dtype: float64

In [16]:
# número de pedidos diferentes com review
order_reviews['order_id'].nunique()

41606

Juntar Review e produtos

Temos o id do pedido. Ver quais produtos estão inseridos no pedido.

In [17]:
# Ver categorias dos produtos

In [18]:
products.sample()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
21885,f2c6bb3c72a67ac37f14bae1faddcf40,beleza_saude,52.0,676.0,1.0,350.0,30.0,12.0,16.0


In [19]:
products['product_category_name'].value_counts().head(15)

cama_mesa_banho                3029
esporte_lazer                  2867
moveis_decoracao               2657
beleza_saude                   2444
utilidades_domesticas          2335
automotivo                     1900
informatica_acessorios         1639
brinquedos                     1411
relogios_presentes             1329
telefonia                      1134
bebes                           919
perfumaria                      868
papelaria                       849
fashion_bolsas_e_acessorios     849
cool_stuff                      789
Name: product_category_name, dtype: int64

In [20]:
products['product_category_name'].unique()

array(['perfumaria', 'artes', 'esporte_lazer', 'bebes',
       'utilidades_domesticas', 'instrumentos_musicais', 'cool_stuff',
       'moveis_decoracao', 'eletrodomesticos', 'brinquedos',
       'cama_mesa_banho', 'construcao_ferramentas_seguranca',
       'informatica_acessorios', 'beleza_saude', 'malas_acessorios',
       'ferramentas_jardim', 'moveis_escritorio', 'automotivo',
       'eletronicos', 'fashion_calcados', 'telefonia', 'papelaria',
       'fashion_bolsas_e_acessorios', 'pcs', 'casa_construcao',
       'relogios_presentes', 'construcao_ferramentas_construcao',
       'pet_shop', 'eletroportateis', 'agro_industria_e_comercio', nan,
       'moveis_sala', 'sinalizacao_e_seguranca', 'climatizacao',
       'consoles_games', 'livros_interesse_geral',
       'construcao_ferramentas_ferramentas',
       'fashion_underwear_e_moda_praia', 'fashion_roupa_masculina',
       'moveis_cozinha_area_de_servico_jantar_e_jardim',
       'industria_comercio_e_negocios', 'telefonia_fixa',
   

# Merge dataset reviews - order_items - products 

In [21]:
order_reviews.sample()
order_items.sample()
products.sample()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,n_char_title,n_char_text,n_sentences
70648,1e4f2c4cd5c91c873c1a035c8b7ab0ee,b40456ac26df93f73f7cac7e9d87de5a,4,NaN,"Faltou o fundo da lixeira, o resto veio tudo c...",2017-07-15 00:00:00,2017-07-18 18:24:24,0,51,1


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
102112,e7c5ccfc34c262f8f9240ade57e01ed7,1,48465725d322101bdadd1d1a5acd8480,980640c45d7a4635885491d077167e4d,2018-08-08 12:50:11,94.99,18.77


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
7484,873417a13086036760bd9528d4d13d27,casa_conforto,41.0,529.0,1.0,800.0,20.0,20.0,20.0


In [22]:
order_reviews.shape
order_reviews['order_id'].nunique()

# em order items há repetição do order_id no caso de pedidos com mais de 
# um pedido
order_items.shape
order_items['order_id'].nunique()

(41753, 10)

41606

(112650, 7)

98666

In [23]:
reviews_product = order_reviews.merge(order_items[['order_id', 'product_id']], on='order_id', how='left')
reviews_product['product_id'].nunique()

19168

In [24]:
# categoria do produto
reviews_product.shape
reviews_product = reviews_product.merge(products[['product_id', 'product_category_name']], on='product_id', how='left')
reviews_product.shape

(49135, 11)

(49135, 12)

In [25]:
reviews_product['product_category_name'].value_counts()

cama_mesa_banho                  5377
beleza_saude                     3834
moveis_decoracao                 3663
esporte_lazer                    3499
informatica_acessorios           3336
                                 ... 
fashion_esporte                     7
cds_dvds_musicais                   5
pc_gamer                            2
seguros_e_servicos                  2
fashion_roupa_infanto_juvenil       1
Name: product_category_name, Length: 73, dtype: int64